In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1730176317.496682      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1029 04:31:57.504799298      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1029 04:31:57.504815402      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1029 04:31:57.504818656      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1029 04:31:57.504821009      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1029 04:31:57.504823431      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1730176333.755173      13 service.cc:145] XLA service 0x5a6fe1f5fe50 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730176333.755243      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1730176333.755249      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1730176333.755252      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1730176333.755255      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1730176333.755257      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1730176333.755277      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1730176333.755280      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1730176333.755283      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 96


In [5]:
Test = True

if Test:
    num_jobs = 92
else:
    num_jobs = 4

config_dict = {'train_csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
config_dict['test_csv_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
config_dict['train_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
config_dict['test_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [6]:
train_data = pl.read_csv(config_dict['train_csv_path'], low_memory=True)
# train_data.describe()
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
# len(train_data_id_list)

# actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk(config_dict['train_parquet_root_path']):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        # actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))
        
final_id_list = list(set(actigraphy_id_list).intersection(train_data_id_list))

len(final_id_list)

996

In [57]:
train_data.head(10)

id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,…,PAQ_C-PAQ_C_Total,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,encoded_Basic_Demos-Enroll_Season,encoded_CGAS-Season,encoded_Physical-Season,encoded_Fitness_Endurance-Season,encoded_FGC-Season,encoded_BIA-Season,encoded_PCIAT-Season,encoded_PreInt_EduHx-Season,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season
str,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""00008ff9""",5,0,51,16.877316,46.0,50.8,null,null,null,null,null,null,null,0,0,null,null,null,null,0,0,7.0,0,6.0,0,6.0,1,2,2.66855,16.8792,932.498,1492.0,8.25598,41.5862,13.8177,3.06143,…,null,5,4,4,0,4,0,0,4,0,0,4,0,4,4,4,4,4,4,2,4,55,null,null,3,2,0,3,0,4,0,0,0,0,4,4,4
"""000fd460""",9,0,null,14.03559,48.0,46.0,22.0,75,70,122,null,null,null,3,0,null,null,null,null,5,0,11.0,1,11.0,1,3.0,0,2,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,…,2.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,64,0,0,2,4,0,4,0,3,0,2,4,0,0
"""00105258""",10,1,71,16.648696,56.5,75.6,null,65,94,117,5,7,33,20,1,10.2,1,14.7,2,7,1,10.0,1,10.0,1,5.0,0,null,null,null,null,null,null,null,null,null,…,2.17,5,2,2,1,2,1,1,2,1,1,1,0,1,1,1,0,2,2,1,1,28,38,54,2,0,2,0,0,0,0,4,0,2,4,2,0
"""00115b9f""",9,0,71,18.292347,56.0,81.6,null,60,97,117,6,9,37,18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,3,3.84191,18.2943,1131.43,1923.44,15.5925,62.7757,14.074,4.22033,…,2.451,4,2,4,0,5,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,31,45,0,1,3,0,2,2,2,2,2,3,4,3,2
"""0016bb22""",18,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,2,4,4,4,4,4,4,2,4,4
"""001f3379""",13,1,50,22.279952,59.5,112.2,null,60,73,102,null,null,null,12,0,16.5,2,17.9,2,6,0,10.0,1,11.0,1,8.0,0,2,4.33036,30.1865,1330.97,1996.45,30.2124,84.0285,16.6877,13.4988,…,4.11,3,3,3,0,2,1,0,2,2,1,0,1,3,3,2,1,3,1,2,1,34,40,56,0,1,1,3,2,4,2,2,2,1,4,1,2
"""0038ba98""",10,0,null,19.66076,55.0,84.6,null,123,83,163,null,null,null,9,1,null,null,null,null,2,0,11.0,1,11.0,1,11.0,1,2,3.78271,19.6629,1135.86,1817.38,16.3275,63.247,14.7,4.96291,…,3.67,1,4,1,0,2,1,0,1,0,0,0,0,0,0,0,4,1,4,1,0,20,27,40,3,0,0,4,0,4,0,0,3,0,4,3,3
"""0068a485""",10,1,null,16.861286,59.25,84.2,27.0,71,90,116,null,null,null,0,0,12.6,2,11.1,1,0,0,0.0,0,0.0,0,4.0,0,3,4.05726,16.8631,1180.04,1888.06,21.94,67.9527,13.6092,3.25395,…,1.27,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,null,0,4,0,4,0,0,4,0,4,0,4
"""0069fbed""",15,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…

In [55]:
train_data.columns

['id',
 'Basic_Demos-Enroll_Season',
 'Basic_Demos-Age',
 'Basic_Demos-Sex',
 'CGAS-Season',
 'CGAS-CGAS_Score',
 'Physical-Season',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Waist_Circumference',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'Fitness_Endurance-Season',
 'Fitness_Endurance-Max_Stage',
 'Fitness_Endurance-Time_Mins',
 'Fitness_Endurance-Time_Sec',
 'FGC-Season',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_GSND',
 'FGC-FGC_GSND_Zone',
 'FGC-FGC_GSD',
 'FGC-FGC_GSD_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'BIA-Season',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_BMC',
 'BIA-BIA_BMI',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_ECW',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'BIA-BIA_ICW',
 'BIA-BIA_LDM',
 'BIA-BIA_LST',
 'BIA-BIA_SMM',
 'BIA-BIA_TBW',
 'PAQ_A-Season',

In [56]:
categorical_colms = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season'\
                    ,'BIA-Season','PCIAT-Season','PreInt_EduHx-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season']

for cat_col in categorical_colms:
    cat_vals_list = pl.Series(train_data.select(pl.col(cat_col))).to_list()
    encoded_vals_list = le.fit_transform(cat_vals_list)
    encoded_col = "encoded_{0}".format(cat_col)
    train_data = train_data.with_columns(pl.Series(name=encoded_col, values=encoded_vals_list))
    train_data = train_data.drop([cat_col])

In [8]:
# Calculate the approximate size of each smaller list
chunk_size = len(final_id_list) // num_jobs
remainder = len(final_id_list) % num_jobs

# Create the num_jobs lists
result = []
start = 0

for i in range(num_jobs):
    # Add an extra element to the first few lists to account for remainder
    end = start + chunk_size + (1 if i < remainder else 0)
    result.append(final_id_list[start:end])
    start = end

# Check the result
for i, sublist in enumerate(result):
    print(f"List {i + 1} (length {len(sublist)}): {sublist}")

List 1 (length 11): ['9cba0d0e', 'b802aec3', '81d3ab22', '432aff11', '5f37ceb1', '83912085', '3ab539f0', '40087bc5', '80b46ffb', '427037b2', '948a7025']
List 2 (length 11): ['f0d76b32', 'a00c85e7', 'f1bd0f59', 'e328a032', 'fdc11b96', '5ee595e8', '641697e6', '9271574b', '8a645723', 'a2994e55', '95cc96cc']
List 3 (length 11): ['d3da09e8', '2f8ec752', '62cd5c74', '8d8b8553', 'da19c97e', '1d78b3c3', '36c060b9', '2d0bbbd3', 'f2d4efd8', '05bbed1b', '1b2eea8f']
List 4 (length 11): ['419d05c6', 'c5cfaeae', '3fcef947', '3826be66', 'e4b47411', '9a2082cf', '8f015125', 'bebff291', '1a803bfe', 'a28c7891', 'ab987c75']
List 5 (length 11): ['a26c3e8f', 'e0db55b1', 'bf31e0a7', 'adaf7461', '836b4de3', 'bea525dc', 'fc2c2c08', '2a0b8386', '2c2dc20d', '2cd9b5a3', 'ecd25dd1']
List 6 (length 11): ['35d2aa41', '2f154763', 'a3bd8858', '4926b18a', 'c9e419e7', 'd2d9b474', 'd6cca65e', '25eba023', '30d2fd47', '137baaec', 'c5462e87']
List 7 (length 11): ['08660d12', '2e17c497', '4e6c7a75', 'b08f0ca4', '83846d0c', '

In [9]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [42]:
def unify_parquet_files(list_of_ids):
    df_initializing_id = list_of_ids.pop()

    parquet_file_name = "id={0}/*.parquet".format(df_initializing_id)
    first_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name

    first_parquet_df = pl.read_parquet(first_parquet_file)
    first_parquet_df = first_parquet_df.drop(['id'])

    first_parquet_df = first_parquet_df.with_columns([
        pl.lit(df_initializing_id).alias("id"),
        pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day")
    ])

    first_parquet_df = first_parquet_df.drop(['time_of_day'])
    first_parquet_df = first_parquet_df.group_by(["id","weekday","hour_of_day","relative_date_PCIAT"])\
    .agg([pl.mean("enmo").name.suffix("_avg"), pl.mean("anglez").name.suffix("_avg"), pl.mean("light").name.suffix("_avg")\
          , pl.mean("battery_voltage").name.suffix("_avg")])
    
    train_data_of_id = train_data.filter(pl.col('id')==df_initializing_id)

    first_joined_features_df = first_parquet_df.join(train_data_of_id, on="id")
    
    for id in tqdm(list_of_ids):
        parquet_file_name = "id={0}/*.parquet".format(id)
        parquet_file_path = config_dict['train_parquet_root_path']+parquet_file_name
        parquet_df = pl.read_parquet(parquet_file_path)
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        parquet_df = parquet_df.group_by(["id","weekday","hour_of_day","relative_date_PCIAT"])\
        .agg([pl.mean("enmo").name.suffix("_avg"), pl.mean("anglez").name.suffix("_avg"), pl.mean("light").name.suffix("_avg")\
              , pl.mean("battery_voltage").name.suffix("_avg")])
        
        train_data_of_id = train_data.filter(pl.col('id')==id)

        joined_features = parquet_df.join(train_data_of_id, on="id")
        
        try:
            first_joined_features_df = pl.concat([first_joined_features_df, joined_features])
        except:
            print(id)
    return first_joined_features_df

In [44]:
parts_of_parquet = (Parallel(n_jobs=num_jobs)(delayed(unify_parquet_files)(id_list) for id_list in result))

100%|██████████| 10/10 [00:41<00:00,  4.18s/it]


In [45]:
len(parts_of_parquet)

92

In [46]:
parts_of_parquet[0].shape

(7272, 89)

In [47]:
test_df = parts_of_parquet[0]
dd.sql("select * from test_df limit 100").pl()

id,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,…,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
str,i8,i32,f32,f32,f32,f32,f32,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,…,f64,f64,f64,f64,f64,str,f64,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64
"""948a7025""",2,11,14.0,0.052321,7.050202,32.756142,4176.574707,"""Winter""",8,1,"""Summer""",55,"""Spring""",20.595386,51.0,76.2,null,69,90,114,"""Spring""",5,8,29,"""Spring""",4,1,null,null,null,null,0,0,11.0,1,11.0,…,23.8646,13.3439,49.8696,22.6391,39.3345,null,null,null,null,"""Spring""",4,2,2,1,1,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,17,"""Spring""",37,53,"""Winter""",1,0
"""948a7025""",3,16,22.0,0.081716,4.311418,11.586094,3975.389893,"""Winter""",8,1,"""Summer""",55,"""Spring""",20.595386,51.0,76.2,null,69,90,114,"""Spring""",5,8,29,"""Spring""",4,1,null,null,null,null,0,0,11.0,1,11.0,…,23.8646,13.3439,49.8696,22.6391,39.3345,null,null,null,null,"""Spring""",4,2,2,1,1,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,17,"""Spring""",37,53,"""Winter""",1,0
"""948a7025""",4,13,23.0,0.079395,-17.294823,15.195121,3956.541748,"""Winter""",8,1,"""Summer""",55,"""Spring""",20.595386,51.0,76.2,null,69,90,114,"""Spring""",5,8,29,"""Spring""",4,1,null,null,null,null,0,0,11.0,1,11.0,…,23.8646,13.3439,49.8696,22.6391,39.3345,null,null,null,null,"""Spring""",4,2,2,1,1,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,17,"""Spring""",37,53,"""Winter""",1,0
"""948a7025""",6,19,25.0,0.09583,-3.659985,15.382941,3917.755371,"""Winter""",8,1,"""Summer""",55,"""Spring""",20.595386,51.0,76.2,null,69,90,114,"""Spring""",5,8,29,"""Spring""",4,1,null,null,null,null,0,0,11.0,1,11.0,…,23.8646,13.3439,49.8696,22.6391,39.3345,null,null,null,null,"""Spring""",4,2,2,1,1,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,17,"""Spring""",37,53,"""Winter""",1,0
"""948a7025""",4,6,30.0,0.067819,-6.543149,47.946636,3822.265625,"""Winter""",8,1,"""Summer""",55,"""Spring""",20.595386,51.0,76.2,null,69,90,114,"""Spring""",5,8,29,"""Spring""",4,1,null,null,null,null,0,0,11.0,1,11.0,…,23.8646,13.3439,49.8696,22.6391,39.3345,null,null,null,null,"""Spring""",4,2,2,1,1,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,17,"""Spring""",37,53,"""Winter""",1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""948a7025""",5,22,31.0,0.097035,8.231213,143.725693,3811.660889,"""Winter""",8,1,"""Summer""",55,"""Spring""",20.595386,51.0,76.2,null,69,90,114,"""Spring""",5,8,29,"""Spring""",4,1,null,null,null,null,0,0,11.0,1,11.0,…,23.8646,13.3439,49.8696,22.6391,39.3345,null,null,null,null,"""Spring""",4,2,2,1,1,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,17,"""Spring""",37,53,"""Winter""",1,0
"""948a7025""",6,16,32.0,0.056333,28.907009,10.232851,3801.854492,"""Winter""",8,1,"""Summer""",55,"""Spring""",20.595386,51.0,76.2,null,69,90,1